In [26]:
import pandas as pd
import numpy as np

In [27]:
futures = pd.read_excel("Crude Oil May 24.xlsx")
futures["Date"] = pd.to_datetime(futures["Date"])
futures

,Date,Open,High,Low,Close*,Adj Close**,Volume
0,2024-03-25,80.85,82.48,80.59,81.95,81.95,231101
1,2024-03-22,80.84,81.45,80.42,80.63,80.63,231101
2,2024-03-21,81.49,81.92,80.30,81.07,81.07,266860
3,2024-03-20,83.20,83.21,81.44,81.68,81.68,351161
4,2024-03-19,82.85,83.85,82.39,83.47,83.47,76814
...,...,...,...,...,...,...,...
1256,2019-04-01,60.24,61.80,60.13,61.59,61.59,670808
1257,2019-03-29,59.53,60.73,59.41,60.14,60.14,705559
1258,2019-03-28,59.40,59.56,58.20,59.30,59.30,712209
1259,2019-03-27,60.03,60.22,58.81,59.41,59.41,732150


In [28]:
spot_cushing = pd.read_csv("Cushing_OK_WTI_Spot_Price_FOB.csv", skiprows=4)
spot_cushing["Day"] = pd.to_datetime(spot_cushing["Day"])
spot_cushing

,Day,Cushing OK WTI Spot Price FOB Dollars per Barrel
0,2024-03-25,82.41
1,2024-03-22,81.10
2,2024-03-21,81.99
3,2024-03-20,82.79
4,2024-03-19,84.39
...,...,...
9624,1986-01-08,25.87
9625,1986-01-07,25.85
9626,1986-01-06,26.53
9627,1986-01-03,26.00


In [29]:
spot_gulf = pd.read_csv("U.S._Gulf_Coast_Kerosene-Type_Jet_Fuel_Spot_Price_FOB.csv", skiprows=4)
spot_gulf["Day"] = pd.to_datetime(spot_gulf["Day"])

# Convert gallons to barells
spot_gulf['U.S. Gulf Coast Kerosene-Type Jet Fuel Spot Price FOB  Dollars per Barell'] = spot_gulf['U.S. Gulf Coast Kerosene-Type Jet Fuel Spot Price FOB  Dollars per Gallon']*42
spot_gulf

,Day,U.S. Gulf Coast Kerosene-Type Jet Fuel Spot Price FOB Dollars per Gallon,U.S. Gulf Coast Kerosene-Type Jet Fuel Spot Price FOB Dollars per Barell
0,2024-03-25,2.583,108.486
1,2024-03-22,2.582,108.444
2,2024-03-21,2.608,109.536
3,2024-03-20,2.606,109.452
4,2024-03-19,2.671,112.182
...,...,...,...
8533,1990-04-06,0.536,22.512
8534,1990-04-05,0.540,22.680
8535,1990-04-04,0.560,23.520
8536,1990-04-03,0.555,23.310


In [30]:
# The time series for futures and kerosene differ in size, we will use an inner join on the dates to ensure a valid correlation calculation
common_dates = pd.Series(list(set(futures['Date']) & set(spot_gulf['Day'])))
spot_gulf_filtered = spot_gulf[spot_gulf['Day'].isin(common_dates)].reset_index(drop=True)
futures_filtered = futures[futures['Date'].isin(common_dates)].reset_index(drop=True)


In [ ]:
# We will use a 125-day window to reflect the market dynamics over the same timeframe to the hedge horizon
hedge_horizon = 125

# Only compute standard deviation and correlation on dates prior to the contract
spot_gulf_november = spot_gulf_filtered[spot_gulf_filtered["Day"] <  "2023-11-15"].copy()
futures_november = futures_filtered[futures_filtered["Date"] <  "2023-11-15"].copy()

spot_diff = spot_gulf_november.set_index("Day")[
    "U.S. Gulf Coast Kerosene-Type Jet Fuel Spot Price FOB  Dollars per Barell"
].diff()

futures_diff = futures_november.set_index("Date")["Close*"].diff()


rolling_sigma_s = spot_diff.rolling(window=hedge_horizon).std()
rolling_sigma_f = futures_diff.rolling(window=hedge_horizon).std()

rolling_rho = spot_diff.rolling(window=hedge_horizon).corr(
    futures_diff
)

rolling_h = (rolling_rho * rolling_sigma_s) / rolling_sigma_f

h = rolling_h.dropna().iloc[-1]

print("The hedge ratio is: ",h)
print("So the company needs a long position in: ",h*2e6/42 ,"gallons of Crude Oil futures to hedge the purchase of 2M gallons of kerosene")
n_contracts = round(h*2e6/42000)
print("Each contract is for 1000 barrels, i.e., 42k gallons so the company requires: ",n_contracts ,"futures contracts")

print("The net price per gallon after hedge is then:",(spot_gulf[spot_gulf["Day"]=="2024-03-19"]['U.S. Gulf Coast Kerosene-Type Jet Fuel Spot Price FOB  Dollars per Barell'].values[0]*2e6/42
      -(futures[futures["Date"]=="2024-03-19"]['Close*'].values[0]
      -futures[futures["Date"]=="2023-11-15"]['Close*'].values[0])*1000*n_contracts)/2e6
      )
print("The spot price is: ", spot_gulf[spot_gulf["Day"]=="2024-03-19"]['U.S. Gulf Coast Kerosene-Type Jet Fuel Spot Price FOB  Dollars per Gallon'].values[0])

# The net hedged price per gallon is below the spot price, the hedging was effective

The hedge ratio is:  0.8866092124214583
So the company needs a long position in:  42219.48630578373 gallons of Crude Oil futures to hedge the purchase of 2M gallons of kerosene
Each contract is for 1000 barrels, i.e., 42k gallons so the company requires:  42 futures contracts
The net price per gallon after hedge is then: 2.5279899999999995
The spot price is:  2.671


In [ ]:
print("Basis Risk is:",(spot_cushing[spot_cushing["Day"]=="2024-03-19"]["Cushing OK WTI Spot Price FOB  Dollars per Barrel"].values[0]
       -spot_gulf[spot_gulf["Day"]=="2024-03-19"]['U.S. Gulf Coast Kerosene-Type Jet Fuel Spot Price FOB  Dollars per Barell'].values[0]
)/42)

# The negative basis risk means the futures underperformed the spot price so the hedge provided less offset than expected.


Basis Risk is: -0.6617142857142854


In [34]:
print("The net profit of the hedge fund is:",(-78
+spot_cushing[spot_cushing["Day"]=="2023-12-08"]["Cushing OK WTI Spot Price FOB  Dollars per Barrel"].values[0])*1e6
+(futures[futures["Date"]=="2022-01-12"]['Close*'].values[0]-futures[futures["Date"]=="2023-12-08"]['Close*'].values[0])*1e6
)

The net profit of the hedge fund is: 4280000.000000001


In [35]:
initial_margin = 6e6
r = 4/100
r_margin = 3/100

filtered_futures = futures[(futures["Date"] >= "2022-01-12") & (futures["Date"] <= "2023-12-09")]


accounts = pd.DataFrame(columns=["Date", "Future Price",  "Margin Account", "Cash Account","PnL"])
accounts["Date"] = filtered_futures["Date"]
accounts["Future Price"] = filtered_futures["Close*"].values 
accounts = accounts.sort_values("Date").reset_index(drop=True)
accounts.loc[accounts["Date"] == "2022-01-12", "Margin Account"] = 6e6
accounts.loc[accounts["Date"] == "2022-01-12", "Cash Account"] = 12e6
accounts["PnL"] = -accounts["Future Price"].diff()*1e6
accounts.loc[accounts["Date"] == "2022-01-12", "PnL"] = 0

for i in range(1,len(accounts),1):
    accounts.loc[i, "Margin Account"] = accounts.loc[i-1, "Margin Account"] + accounts.loc[i, "PnL"]
    if accounts.loc[i, "Margin Account"]<6e6:
        missing = 6e6-accounts.loc[i, "Margin Account"]
        print( missing)

        accounts.loc[i, "Margin Account"] = 6e6
        accounts.loc[i, "Cash Account"] = accounts.loc[i-1, "Cash Account"] -missing
    else:
        accounts.loc[i, "Cash Account"] = accounts.loc[i-1, "Cash Account"]
        

    accounts.loc[i, "Margin Account"] = accounts.loc[i, "Margin Account"]*(1+r_margin/365)
    accounts.loc[i, "Cash Account"] = accounts.loc[i, "Cash Account"]*np.exp(r/365)

    if accounts.loc[i, "Cash Account"]<0:
        print("Position liquidated on day: ", accounts.loc[i,"Date"],", missing: USD",abs(accounts.loc[i, "Cash Account"]))
        print("We then invest the remainings at 4%: ")
        print("Hedge fund capital on december 9, 2023: USD",(accounts.loc[i, "Cash Account"]+accounts.loc[i, "Margin Account"])*np.exp(r*(pd.to_datetime("2023-12-09") - pd.to_datetime(accounts.loc[i,"Date"])).days/365))
        break



1179464.1095890328
1609506.8493150817
1529506.8493150547
386967.4893782176
798416.0377823785
49506.84931506496
59506.849315069616
2009506.8493150584
2039506.8493150743
786792.3935231501
2359506.8493150673
Position liquidated on day:  2022-02-14 00:00:00 , missing: USD 794087.6330913511
We then invest the remainings at 4%: 
Hedge fund capital on december 9, 2023: USD 5598771.686915335


In [ ]:
print("If the hedge fund manager had only taken the long position in the forward contract the hedge fund's total capital on 9 December 2023 would have been:", 18e6*np.exp(r*(pd.to_datetime("2023-12-09") - pd.to_datetime("2022-01-12")).days/365)
      -78e6
      +spot_cushing[spot_cushing["Day"]=="2023-12-08"]["Cushing OK WTI Spot Price FOB  Dollars per Barrel"].values[0]*1e6
      )

# Despite the apparent arbitrage opportunity, the hedge fund lost nearly $6.7 million of potential capital by hedging the forward position.

If the hedge fund manager had only taken the long position in the forward contract the hedge fund's total capital on 9 December 2023 would have been: 12296647.975114256
